In [25]:
from elasticsearch import Elasticsearch
import requests

# Configuración
es = Elasticsearch("http://elasticsearch:9200")
index_name = "asignaturas_prueba"
graphdb_repo = "http://graphdb:7200/repositories/asignaturas"
auth = None  # ('usuario','password') si hace falta

# Escapar literales RDF
def escape_rdf_literal(text):
    if text is None:
        return ""
    return text.replace('\\', '\\\\').replace('"', '\\"').replace('\n', '\\n')

# Convertir doc a triples
def doc_to_triples(doc):
    subj = f"<http://upm.es/ontology/asignatura/{doc['id_asignatura']}>"
    triples = []

    triples.append(f'{subj} <http://upm.es/ontology/descripcion> "{escape_rdf_literal(doc.get("descripcion_asignatura",""))}" .')
    triples.append(f'{subj} <http://upm.es/ontology/conocimientosPrevios> "{escape_rdf_literal(doc.get("conocimientos_previos",""))}" .')

    for comp in doc.get("competencias", []):
        triples.append(f'{subj} <http://upm.es/ontology/competencia> "{escape_rdf_literal(comp.get("texto",""))}" .')
        triples.append(f'{subj} <http://upm.es/ontology/codigoCompetencia> "{escape_rdf_literal(comp.get("codigo",""))}" .')

    for tema in doc.get("temario", []):
        triples.append(f'{subj} <http://upm.es/ontology/tema> "{escape_rdf_literal(tema.get("titulo",""))}" .')
        for sub in tema.get("subtemas", []):
            triples.append(f'{subj} <http://upm.es/ontology/subtema> "{escape_rdf_literal(sub.get("titulo",""))}" .')

    return triples

# Leer de Elasticsearch
query = {"match_all": {}}
res = es.search(index=index_name, query=query, size=1000)

all_triples = []
for hit in res['hits']['hits']:
    doc = hit['_source']
    all_triples.extend(doc_to_triples(doc))

# Subir a GraphDB por batch
batch_size = 20
for i in range(0, len(all_triples), batch_size):
    batch = all_triples[i:i+batch_size]
    sparql_update = "INSERT DATA { " + "\n".join(batch) + " }"
    r = requests.post(
        f"{graphdb_repo}/statements",
        data={"update": sparql_update},
        auth=auth
    )
    if r.status_code != 204:
        print("Error subiendo batch:", r.status_code, r.text)
    else:
        print(f"Batch {i//batch_size + 1} subido correctamente")




Batch 1 subido correctamente
Batch 2 subido correctamente
Batch 3 subido correctamente


In [24]:
import requests

graphdb_repo = "http://graphdb:7200/repositories/asignaturas"
query = """
PREFIX ex: <http://example.org/schema/>
SELECT ?asignatura ?competencia
WHERE {
  ?asignatura ex:competencia ?competencia .
}
LIMIT 10
"""

r = requests.get(graphdb_repo, params={"query": query})
print(r.text)



asignatura,competencia
http://example.org/asignatura/613000129,Que los estudiantes posean las habilidades de aprendizaje que les permitan continuar estudiando de un modo que habrá de ser en gran medida autodirigido o autónomo.
http://example.org/asignatura/613000129,"Poseer y comprender conocimientos que aporten una base u oportunidad de ser originales en el desarrollo y/o aplicación de ideas, a menudo en un contexto de investigación"
http://example.org/asignatura/613000129,Que los estudiantes sepan aplicar los conocimientos adquiridos y su capacidad de resolución de problemas en entornos nuevos o poco conocidos dentro de contextos más amplios (o multidisciplinares) relacionados con su área de estudio
http://example.org/asignatura/613000129,"Que los estudiantes sean capaces de integrar conocimientos y enfrentarse a la complejidad de formular juicios a partir de una información que, siendo incompleta o limitada, incluya reflexiones sobre las responsabilidades sociales y éticas vinculada